In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import os
import os.path as op
import sys
import itertools
#from gensim.matutils import softcossim

In [ ]:
df = pd.read_csv('reddit_posts_2022_07_21-10_16_58_AM.csv', index_col=0)
df.reset_index(inplace=True)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the vectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
# fit_transform applies TF-IDF to clean texts - we save the array of vectors in X
X_raw_tfidf = vectorizer.fit_transform(df['selftext'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27216 entries, 0 to 27215
Data columns (total 83 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   index                          27216 non-null  int64  
 1   _id                            27216 non-null  object 
 2   all_awardings                  27216 non-null  object 
 3   allow_live_comments            27216 non-null  object 
 4   author                         27216 non-null  object 
 5   author_flair_css_class         0 non-null      float64
 6   author_flair_richtext          27212 non-null  object 
 7   author_flair_text              1425 non-null   object 
 8   author_flair_type              27212 non-null  object 
 9   author_fullname                27212 non-null  object 
 10  author_is_blocked              18096 non-null  object 
 11  author_patreon_flair           27212 non-null  object 
 12  author_premium                 27212 non-null 

In [9]:
from sklearn.metrics.pairwise import cosine_similarity

#def cosine_similarity(a, b):
#    return np.inner(a,b)/(norm(a)*norm(b))

def delete_similar_text_from_df(df):
    if df.shape[0] == 1:
        return df
    else:
        tf_idf_vectors = vectorizer.fit_transform(df['selftext'])
        all_indices = df.index.values.tolist()
        all_combinations = itertools.combinations(all_indices, 2)
        new_indices = []
        
        while(len(all_indices) > 1):
            
            first = all_indices[0]
            rest = all_indices[1::]
            
            vec1 = tf_idf_vectors[first]
        
            for i in rest:
                vec2 = tf_idf_vectors[i]
                cossim = cosine_similarity(vec1, vec2)
                
                if (cossim > 0.98):
                    all_indices.remove(i)
                    
            new_indices.append(first)
            all_indices.remove(first)

        else:
            if len(all_indices) > 0:
                new_indices.append(all_indices[0])
        return df.iloc[new_indices]


In [ ]:
new_df = delete_similar_text_from_df(df)

In [23]:
# exporting the dataset without content based duplicates into csv
new_df.drop(['removed_by_category', 'author_flair_template_id', 'media_metadata'], axis=1).to_csv("reddit_posts_2022_07_21-10_16_58_AM_no_duplicates.csv")
n_df = pd.read_csv("reddit_posts_2022_07_21-10_16_58_AM_no_duplicates.csv", index_col=0)